# Code for Question 5 and Question 6

In [1]:
from copy import copy
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
theta = 0.5
# observation probability does not change according to original action.
# if we believe the person is at home, then the probability of show up at 1 interval is theta
# O(show| home) = theta, O(!show|home)=1-theta
# if we beleive the person is not at home, then O(show|!home) = 0 and O(!show|!home) = 1
observation = np.asarray([[theta,1-theta],[0,1]])
# transition does not change acorrding to action
transition = np.asarray([[1,0],[0,1]])
#Basic data strucutre to hold information
class Home():
    index = None
    light = False
    def __init__(self, index, light):
        self.index, self.light = index, light
    def copy(self):
        return self.index, self.light
    def __repr__(self):
        return "House " + str(self.index)+ " with light " + str(self.light)
class Node():
    parent = None
    state = None
    level = 0
    left, right = None, None
    def __init__(self,state, level):
        self.state, self.level = state, level
        if self.state.light:
            self.belief =  [0.85, 0.15]
        else:
            self.belief = [0.05, 0.95]
    def __repr__(self):
        return str(self.state) + " with belief of " + str(self.belief)
h = Home(1,True)
root = Node(h, 0)
queue = [(2, True), (3, False), (4, False), (5,False)]
# count stands for how many times the current node has been used/waited
# max_count stands for how many times the node can be repreaeted
def build(node, queue, depth, count=100, max_count = 100):
    if len(queue)> 0 and depth>0:
        if count>0:
    # left means no show and wait, state transition is st+1=st
            node.left = Node(Home(*node.state.copy()), node.level+1)
            node.left.parent = node
            build(node.left, queue, depth -1, count -1, max_count)
    # right show and move, state transistion is st+1 = next house in queue
        node.right = Node(Home(*queue[0]), node.level+1)
        node.right.parent = node
        build(node.right, queue[1:], depth-1, max_count-1, max_count)

def belief_update(node):
    if node.left:
        belief = np.sum(observation*transition*node.belief,axis=1)
        belief=belief/np.sum(belief)
        node.left.belief = belief
        belief_update(node.left)
    if node.right:
        belief_update(node.right)
# utility
def utility (node):
    # left + right
    # the left is no show, based on current belief
    # the right is show, based on current belieft, it is theta * belief(home)
    retval = 0
    if node.left:
        retval += (utility(node.left))*(1-theta*node.belief[0]) 
    if node.right:
        retval += (1+utility(node.right))*theta*node.belief[0]        

    return retval

In [2]:
build(root,queue, 18)
belief_update(root)
print(utility(root))

1.61034513102


# Q5 Final Answer is 1.61034513102

In [3]:
build(root,queue, 18,2,3)
belief_update(root)
print(utility(root))

1.32217378082


# Q6 Final Answer is 1.32217378082

In [4]:
build(root,queue, 2)
def get_children(node):
    print(node.left)
    print(node.right)
    if node.left:
        get_children(node.left)
    else:
        print("Ends")
    if node.right:
        get_children(node.right)
    else:
        print("Ends")
get_children(root)
# verify update belief code
belief = [0.85, 0.15]
show = False
belief = np.sum(observation*transition*belief,axis=1)
belief=belief/np.sum(belief)
print(belief)
    
# verify tree and utility calculation
build(root,queue, 1)
print(root)
print(root.left)
print(root.right)
print(utility(root))
print(utility(root.left))
print(utility(root.right))

# verify
build(root,queue, 2)
belief_update(root)
print(root)
print(root.right)
print(root.right.right)
print(root.left)
print(root.left.left)
print(utility(root))
print(utility(root.left))
print(utility(root.right))

House 1 with light True with belief of [0.85, 0.15]
House 2 with light True with belief of [0.85, 0.15]
House 1 with light True with belief of [0.85, 0.15]
House 2 with light True with belief of [0.85, 0.15]
None
None
Ends
Ends
None
None
Ends
Ends
House 2 with light True with belief of [0.85, 0.15]
House 3 with light False with belief of [0.05, 0.95]
None
None
Ends
Ends
None
None
Ends
Ends
[ 0.73913043  0.26086957]
House 1 with light True with belief of [0.85, 0.15]
House 1 with light True with belief of [0.85, 0.15]
House 2 with light True with belief of [0.85, 0.15]
0.425
0
0
House 1 with light True with belief of [0.85, 0.15]
House 2 with light True with belief of [0.85, 0.15]
House 3 with light False with belief of [0.05, 0.95]
House 1 with light True with belief of [ 0.73913043  0.26086957]
House 1 with light True with belief of [ 0.5862069  0.4137931]
0.818125
0.369565217391
0.425
